In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import math
import warnings
warnings.filterwarnings("ignore")


In [2]:
msa_id=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/msa_identifier.dta')
sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')
msa_vars=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/second_stage_dataset_cl.dta')
coefs=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/results/age2_sig_coefs.dta')



In [3]:
logit_results=pd.read_excel('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/results/1st_stage_28_age2.xlsx')
                            
                            
                            

In [4]:
coef=logit_results[logit_results['type']=='point estimate']
se=logit_results[logit_results['type']=='std. err.']
p=logit_results[logit_results['type']=='p-value']

In [5]:
reg=coef.merge(se, on='variable').merge(p,on='variable')

In [6]:
reg=reg[['variable','value','value_x','value_y','type','type_x','type_y']]

In [7]:
reg=reg.rename(columns={'variable':'Variable','value_x':'Coefficient','value_y':'Std. Err.','value':'P-value'})

In [8]:
reg=reg[['Variable','Coefficient','Std. Err.','P-value']]

In [9]:
coefs.melt()

,variable,value
0,htem,28
1,age,age2
2,_b_lnrho,0.050791
3,_b_newid,1.0
4,_b_cold_ca,0.015151
5,_b_cold_coll,-0.004497
6,_b_cold_mw,0.00685
7,_b_cold_ne,-0.004756
8,_b_hot_ca,0.027025
9,_b_hot_coll,-0.004596


In [10]:
reg['Coefficient']=reg['Coefficient'].apply(lambda x: str.format("{:0_.3f}", x))
reg['Std. Err.']=reg['Std. Err.'].apply(lambda x: str.format("{:0_.3f}", x))
reg['P-value.']=reg['P-value'].apply(lambda x: str.format("{:0_.3f}", x))


reg['Coefficient']=np.where(~(reg['Variable'].isin(['Cold_West','d_r1_kid'])),reg['Coefficient']+"***",reg['Coefficient'])

reg['Coefficient']=reg['Coefficient'].replace("_","")
del reg['P-value']
del reg['P-value.']
print(reg.to_latex(index=False, column_format='rrrr'))



\begin{tabular}{rrrr}
\toprule
      Variable & Coefficient & Std. Err. \\
\midrule
       inc\_hat &    0.123*** &     0.015 \\
        Hot\_CA &    0.027*** &     0.001 \\
      Hot\_West &    0.004*** &     0.001 \\
   Hot\_Midwest &    0.012*** &     0.001 \\
 Hot\_Northeast &    0.004*** &     0.001 \\
   Hot\_College &   -0.005*** &     0.000 \\
       Cold\_CA &    0.015*** &     0.001 \\
     Cold\_West &      -0.001 &     0.001 \\
  Cold\_Midwest &    0.007*** &     0.001 \\
Cold\_Northeast &   -0.005*** &     0.001 \\
  Cold\_College &   -0.004*** &     0.000 \\
           d\_s &   -2.626*** &     0.015 \\
          d\_r1 &   -1.078*** &     0.022 \\
          d\_r2 &   -0.590*** &     0.021 \\
       d\_s\_kid &   -0.469*** &     0.024 \\
      d\_r1\_kid &      -0.029 &     0.035 \\
      d\_r2\_kid &   -0.117*** &     0.033 \\
\bottomrule
\end{tabular}



In [49]:
test=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/age2_le0_shock0_v2_uw.dta')




In [11]:
acs=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/logit-ready_full_age2.dta')
acs_lm=acs[acs['chosen']==1]

In [9]:
####get breakouts by college educated

In [10]:
acs_coll=acs_lm.groupby(['abbrev','coll'],as_index=False).agg({'chosen':sum})
acs_coll

KeyError: 'abbrev'

In [79]:
acs_coll

,abbrev,coll,chosen
0,AL,0.0,357.0
1,AL,1.0,983.0
2,AR,0.0,147.0
3,AR,1.0,384.0
4,AZ,0.0,379.0
...,...,...,...
83,VA,1.0,2984.0
84,WA,0.0,550.0
85,WA,1.0,2309.0
86,WI,0.0,276.0


### Heat change chart

In [12]:
temp=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/age2_le0_shock0_v2_uw.dta')
temp=temp.merge(sl,on='statefip')            
temp_lm=temp.drop_duplicates(subset=['abbrev'])[['statefip','abbrev','fexthot_28','fextcold']].sort_values(by='fexthot_28',ascending=False)  

temp_lm['Extreme Heat Day $\Delta$']=temp_lm['fexthot_28'].apply(lambda x: str.format("{:0_.2f}", x))
temp_lm['Extreme Cold Day $\Delta$']=temp_lm['fextcold'].apply(lambda x: str.format("{:0_.2f}", x))
del temp_lm['fexthot_28']
del temp_lm['fextcold']
# del temp_lm['statefip']
#print(temp_lm.to_latex(index=False, column_format='rrr'))



In [13]:
bad_list=['abbrev_x',
 'Extreme Hot Day Change_x',
 'Extreme Cold Day Change_x',
 'abbrev_y',
 'Extreme Hot Day Change_y',
 'Extreme Cold Day Change_y',
 'abbrev_x',
 'Extreme Hot Day Change_x',
 'Extreme Cold Day Change_x',
 'abbrev_y',
 'Extreme Hot Day Change_y',
 'Extreme Cold Day Change_y',
 'abbrev_x',
 'Extreme Hot Day Change_x',
 'Extreme Cold Day Change_x',
 'abbrev_y',
 'Extreme Hot Day Change_y',
 'Extreme Cold Day Change_y']

In [14]:
list(temp_lm)

['statefip',
 'abbrev',
 'Extreme Heat Day $\\Delta$',
 'Extreme Cold Day $\\Delta$']

In [16]:
####tabulate by state college kid, and birth region
acs_lm=acs[acs['chosen']==1]

###get birth region
acs_lm['clizone2'] = np.where(acs_lm.clizone==1,'South','')
acs_lm['clizone2'] =  np.where(acs_lm.clizone==2,'California',acs_lm['clizone2'])
acs_lm['clizone2'] =  np.where(acs_lm.clizone==3,'West',acs_lm['clizone2'])
acs_lm['clizone2'] =  np.where(acs_lm.clizone==4,'Mid West',acs_lm['clizone2'])
acs_lm['clizone2'] =  np.where(acs_lm.clizone==5,'Northeast',acs_lm['clizone2'])

####merge temperature changes
acs_lm=acs_lm[[x for x in acs_lm if x not in bad_list]]

acs_lm=acs_lm.merge(msa_vars[['msa','statefip']],on='msa').merge(temp_lm,on='statefip',how='inner')

# acs_lm_coll=acs_lm.groupby(['abbrev','coll','kid','clizone2'],as_index=False).agg({'chosen':sum})

###college breakouts
acs_lm_c=acs_lm.groupby(['abbrev','coll','clizone2'],as_index=False).agg({'chosen':sum,'Extreme Heat Day $\Delta$':max,'Extreme Cold Day $\Delta$':max})
acs_lm_cs=acs_lm.groupby(['abbrev','coll'],as_index=False).agg({'chosen':sum})


acs_lm_cs=acs_lm_cs.rename(columns={'chosen':'total'})
acs_lm_c=acs_lm_c.merge(acs_lm_cs,on=['abbrev','coll'])
acs_lm_c['Pct.']=acs_lm_c['chosen']/acs_lm_c['total']

acs_lm_c['State']=acs_lm_c['abbrev']
acs_lm_c['Worker Type']=np.where(acs_lm_c['coll']==1,'Skilled','Unskilled')
acs_lm_c['CZ Born In']=acs_lm_c['clizone2']
acs_lm_c=acs_lm_c[['State','Worker Type','CZ Born In','Pct.']]
####limit to FL, UT, OR, TX

for state in ['FL','UT','OR','TX','WA','CA','MT','MS','CO','LA']:
    tmp=acs_lm_c[acs_lm_c['State']==state]
    tmp['Pct.']=tmp['Pct.'].apply(lambda x: str.format("{:.2%}", x))
    tmp=tmp.sort_values(by=['Worker Type','Pct.'],ascending=False)
    print(tmp.to_latex(index=False, column_format='rrrr'))
    

\begin{tabular}{rrrr}
\toprule
State & Worker Type & CZ Born In &   Pct. \\
\midrule
   FL &   Unskilled &      South & 61.23\% \\
   FL &   Unskilled &       West &  2.45\% \\
   FL &   Unskilled & California &  2.20\% \\
   FL &   Unskilled &  Northeast & 19.09\% \\
   FL &   Unskilled &   Mid West & 15.03\% \\
   FL &     Skilled &      South & 54.76\% \\
   FL &     Skilled &       West &  3.46\% \\
   FL &     Skilled &  Northeast & 24.13\% \\
   FL &     Skilled & California &  2.54\% \\
   FL &     Skilled &   Mid West & 15.12\% \\
\bottomrule
\end{tabular}

\begin{tabular}{rrrr}
\toprule
State & Worker Type & CZ Born In &   Pct. \\
\midrule
   UT &   Unskilled &       West & 81.22\% \\
   UT &   Unskilled & California &  8.92\% \\
   UT &   Unskilled &   Mid West &  5.63\% \\
   UT &   Unskilled &  Northeast &  2.35\% \\
   UT &   Unskilled &      South &  1.88\% \\
   UT &     Skilled & California &  8.01\% \\
   UT &     Skilled &       West & 79.92\% \\
   UT &     Skilled &

In [63]:
####kids table
kids_table=acs_lm.groupby(['coll'],as_index=False).agg({'chosen':sum,'kid':sum})


In [64]:
kids_table

,coll,chosen,kid
0,0.0,17090.0,10679.0
1,1.0,62910.0,21476.0


In [ ]:
acs_lm_c['Percent of Worker Type Born in Region'].unique()

In [ ]:
###get sums by state

In [ ]:
acs[acs['d_s']==1]

In [ ]:
list(acs)

In [ ]:
#####tabulate each state by 

In [ ]:
temp_lm=temp.drop_duplicates()

In [ ]:
temp['fexthot_28'].describe()

In [ ]:
list(temp)

In [ ]:
msa_id=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/msa_identifier.dta')



In [ ]:
np.log(.04)

In [ ]:
fe[fe['msa']==168][[x for x in fe if "ln" in x]]

In [ ]:
msa_id[msa_id['metarea'].str.contains('chicago')]

### This routine handles these steps:
#### 1) read in projection data
#### 2) merge with population data for each group
#### 3) converge age groups to total population
#### 4) get update population shares
#### 5) read into gams , run gams 
#### 6) re-import gams results (housing prices, wages, and updates population shares (from logit)
#### 7) prep gams results for incorporation into logit routine
#### 8) re-run shares in stata
#### 9) map out migration flows for top entry exit


### Steps 1-5
##### (1) read in projection data, (2) merge pop data fro each group, (3) translate age group and skill changes to total population pct change (4) get updated gams shares (5) read into gams, and run new counter factual 1 time

In [ ]:
# tmp0_gr=tmp0.groupby(['msa','statefip'],as_index=False).agg({'id':sum})
# bmk0_gr=bmk0.groupby(['msa'],as_index=False).agg({'id':sum})
# msaid_state_lookup=tmp0_gr[['msa','statefip']].merge(bmk0_gr[['msa']],on='msa').groupby(['msa','statefip'],as_index=False).sum()
# msaid_state_lookup.to_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')

#####get MSA Identifier information
# msa_id=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/msa_identifier.dta')
# msa_vars=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/second_stage_dataset_cl.dta')
state_split=pd.read_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')


In [ ]:
state_age_shares=pd.read_excel('/Users/hannahkamen/Downloads/state_age_shares.xlsx')

In [ ]:
state_age_shares.head()

In [ ]:
###these are the population shares changes from the first cycle of GAMS data inputs (after initial logit shock with temp)
###the shares caluclated here represent the population change from the previous iteration 
#state_split=pd.read_excel('/Users/hannahkamen/Downloads/state_pop_educ_shares.xlsx')

# msaid_state_lookup=pd.read_excel('/Users/hannahkamen/Downloads/msaid_state_lookup.xlsx')
sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')


for a in [2]:
    for i in [0]:       
#         try:
#             del tmp0
#         except:
#             print("first loop")
        
        #(1) read in projection data, and benchmark population shares

        tmp0=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/projection_data_age2_%s_wbmk_iter5.dta'%i)
        #############merge state lookup onto both projections 
        #sl=pd.read_excel('/Users/hannahkamen/Downloads/statelookup2.xlsx')
        #tmp0=tmp0.merge(sl,on='statefip',how='inner')

        #(2) merge pop share estimates from each group

        ########group benchmark and projected by state, sum shares and chosen to get pct change
        tmp=tmp0.groupby(['state'],as_index=False).agg({'fexthot_28':max,'fextcold':max,'share_it12':sum,'share_it1':sum,'share':sum,'share_b':sum,'share_it13':sum,'share_it14':sum,'share_it15':sum})

        ########merge grouped projections with benchmark to get pct change
        tmp['pct_change']=(tmp['share_it15']-tmp['share_it14'])/tmp['share_it14']

        ########merge pct changes onto population by age

#         tmp=tmp.merge(state_age_shares,on='state',how='inner')
#         tmp['age_id']= a
#         tmp['educ_id']=i
#         ########SET AGE GROUP TWO TO 100 PERCENT OF POPULATION FOR NOW
#         #tmp['contribution_to_total_change']=tmp['pct_change']*tmp['%s'%a]
#         tmp['contribution_to_total_change']=tmp['pct_change']*1
        
#         ######## append all age and skill datasets
#         master=master.append(tmp)






In [ ]:
80000/17090

In [ ]:
tmp0['id'].head(10000)

In [ ]:
tmp0[tmp0['msa']==1]['id'].max()*.25


In [ ]:
tmp=tmp.merge(state_age_shares,on='state',how='inner')
tmp['age_id']= a
tmp['educ_id']=i
########SET AGE GROUP TWO TO 100 PERCENT OF POPULATION FOR NOW
#tmp['contribution_to_total_change']=tmp['pct_change']*tmp['%s'%a]
tmp['contribution_to_total_change']=tmp['pct_change']*1

######## append all age and skill datasets
master=master.append(tmp)


In [ ]:
#(3) translate age group to total population pct change
master_gr=master.groupby(['state','educ_id'],as_index=False).agg({'contribution_to_total_change':sum})

#(4) export to gams format
master_gr['educ_id']=master_gr['educ_id'].astype(str).str.replace('0','unskl').str.replace('1','skl')
master_gr=master_gr.merge(sl,on='state',how='inner')
master_gr=master_gr.rename(columns={'abbrev':'','educ_id':'sk','contribution_to_total_change':'skill_shr'})

master_gr_lm=master_gr[['','sk','skill_shr']]
master_gr_lm.to_csv('/Users/hannahkamen/Downloads/le0_shock_0_it5.csv',index=False)
master.to_excel('/Users/hannahkamen/Downloads/iteration_34_state_shares.xlsx')


In [ ]:
master['educ_id'].unique()

In [ ]:
master_gr['skill_shr'].describe()

#### (6) re-import GAMS results, merge with pop changes

In [ ]:
##### import GAMS results
#y_rpt=pd.read_csv('/Users/hannahkamen/Downloads/y_rpt.csv')
phou_rpt=pd.read_csv('/Users/hannahkamen/Downloads/phou_rpt5.csv')
npl_rpt=pd.read_csv('/Users/hannahkamen/Downloads/npl_rpt5.csv')
####msa pop data
#msa=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/second_stage_dataset_cl.dta')
w_rpt=pd.read_csv('/Users/hannahkamen/Downloads/w_rpt5.csv')


In [ ]:
# w_rpt=w_rpt[['region','skill','household','value']]
# w_rpt=w_rpt.reset_index()
# w_rpt.pivot(index='region',columns=['skill','household'],values='value')

In [ ]:
# test_wage=pd.read_csv('/Users/hannahkamen/Downloads/npl_rpt1.csv')
# test_ph=pd.read_csv('/Users/hannahkamen/Downloads/phou_rpt1.csv')

In [ ]:
####create table with percent of total state that each MSA contributes to
# msa=msa[['statefip','msa','lnpop']]
# msa['msa_pop']=np.exp(msa['lnpop'])
# msa_tot=msa.groupby('statefip',as_index=False).agg({'msa_pop':sum})
# msa_tot=msa_tot.rename(columns={'msa_pop':'msa_pop_total'})
# msa=msa.merge(msa_tot,on='statefip')
# msa['pct_state_total']=msa['msa_pop']/msa['msa_pop_total']
# msa.to_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/msa_pop_pct.dta')


In [ ]:
npl_rpt=npl_rpt[['region','skill','value']]
npl_rpt=npl_rpt.rename(columns={'region':'abbrev','value':'pl'})

In [ ]:
phou_rpt=phou_rpt[['pct','region']]
phou_rpt=phou_rpt.rename(columns={'region':'ph','pct':'abbrev'})

In [ ]:
#####import state pop shares by skilled and unskilled
state_educ=pd.read_excel('/Users/hannahkamen/Downloads/state_educ_shares.xlsx')
del state_educ['statefip']
state_educ=state_educ.merge(sl,on='state',how='inner')
state_educ=state_educ[['statefip','skl','unskl','state','abbrev','state_pop']]

In [ ]:
###merge all fields
r_df=phou_rpt.merge(npl_rpt, on='abbrev').merge(master_gr, left_on=['abbrev','skill'], right_on=['','sk']).merge(state_educ,on=['abbrev','statefip','state'])



In [ ]:
####pivot skill pct changes
pop_changes=r_df.pivot(index='state',columns='sk',values='skill_shr').reset_index()
pop_changes=pop_changes.rename(columns={'skl':'skl_pct_delta','unskl':'unskl_pct_delta'})

#####pivot labor prices
pl_changes=r_df.pivot(index='state',columns='sk',values='pl').reset_index()
pl_changes=pl_changes.rename(columns={'skl':'pl_skl','unskl':'pl_unskl'})

In [ ]:
#r_df=r_df[['statefip','abbrev','state','skill','pl','ph','skl','unskl','state_pop']]
r_df=r_df.merge(pop_changes,on='state').merge(pl_changes,on='state')
r_df['skilled_level_change']=(r_df['skl']*r_df['state_pop']) + (r_df['skl']*r_df['state_pop']*r_df['skl_pct_delta'])
r_df['unskilled_level_change']=(r_df['unskl']*r_df['state_pop']) + (r_df['unskl']*r_df['state_pop']*r_df['unskl_pct_delta'])
r_df['new_state_pop']=r_df['unskilled_level_change']+r_df['skilled_level_change']



In [ ]:
r_df_lm=r_df.drop_duplicates(subset='state')

In [ ]:
del r_df_lm['']
del r_df_lm['new_state_pop']
del r_df_lm['pl']

In [ ]:
r_df_lm2=r_df_lm[['statefip','abbrev','skill','pl_skl','pl_unskl','ph']]

In [ ]:
r_df_lm2.to_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/gams_dta4.dta')




##### (9) map out migration flows

In [ ]:
###prepare dataset with centroid of every state

map_dta1=pd.read_csv('/Users/hannahkamen/Downloads/census_texas.csv')
map_dta2=pd.read_csv('/Users/hannahkamen/Downloads/census_ny.csv')
map_dta=map_dta1.append(map_dta2).reset_index()

###rename data
map_dta['centroid2']=map_dta['centroid1']+","+map_dta['centroid2']
map_dta['centroid1']=map_dta['FULL1_NAME']+","+map_dta['FULL2_NAME']
map_dta['FULL1_NAME']=map_dta['GEOID1']
map_dta['FULL2_NAME']=map_dta['GEOID2']

del map_dta['GEOID1']
del map_dta['GEOID2']


map_dta['GEOID1']=map_dta['level_1']
map_dta['GEOID2']=map_dta['Unnamed: 0']
map_dta=map_dta[['GEOID1','GEOID2','FULL1_NAME','FULL2_NAME','centroid1','centroid2']]
####get abbreviations

states=list(sl['abbrev'].unique())

state_coords=pd.DataFrame()
for s in states:
    tmp=map_dta[map_dta['FULL2_NAME'].str.contains(s)]
    tmp['state']=s
    state_coords=state_coords.append(tmp)
state_coords=state_coords.drop_duplicates(subset='state')
state_coords=state_coords[['state','FULL2_NAME','centroid2']]

In [ ]:
#get sum of shares in all other states for people who chose states 1-44
#get ids that live in state s currentlly
master_map_unskl=pd.DataFrame()
for s in tmp0['state'].unique():
    living_now=[]
    tmp1=tmp0[(tmp0['state']==s) & (tmp0['chosen']==1)]
    ####list of ids currently living in state s
    living_now.append(list(tmp1['id'].unique())[0])
    ###limit dataframe to the ids of people living in state s
    tmp2=tmp0[tmp0['id'].isin(living_now)]
    ####now groupby shares across all states
    tmp3=tmp2.groupby(['state'],as_index=False).agg({'share_it15':sum,'share_b':sum})
    ###tag origin state
    tmp3['living_flag']=s
    master_map_unskl=master_map_unskl.append(tmp3)
    
######merge with state populaiton
master_map0_unskl=master_map_unskl.merge(state_split,left_on='living_flag',right_on='state',how='inner')
master_map0_unskl=master_map0_unskl.rename(columns={'state_x':'moving_to'})

######get share difference between current  iteration and benchmark
master_map0_unskl['share_diff']=master_map0_unskl['share_it15']-master_map0_unskl['share_b']




In [ ]:
master_map0_unskl.to_excel('/Users/hannahkamen/Downloads/map_data_unskl.xlsx')

In [ ]:
#get sum of shares in all other states for people who chose states 1-44
#get ids that live in state s currentlly
master_map_skl=pd.DataFrame()
for s in tmp0['state'].unique():
    living_now=[]
    tmp1=tmp0[(tmp0['state']==s) & (tmp0['chosen']==1)]
    ####list of ids currently living in state s
    living_now.append(list(tmp1['id'].unique())[0])
    ###limit dataframe to the ids of people living in state s
    tmp2=tmp0[tmp0['id'].isin(living_now)]
    ####now groupby shares across all states
    tmp3=tmp2.groupby(['state'],as_index=False).agg({'share_it15':sum,'share_b':sum})
    ###tag origin state
    tmp3['living_flag']=s
    master_map_skl=master_map_skl.append(tmp3)
    
######merge with state populaiton
master_map0_skl=master_map_skl.merge(state_split,left_on='living_flag',right_on='state',how='inner')
master_map0_skl=master_map0_skl.rename(columns={'state_x':'moving_to'})

######get share difference between current  iteration and benchmark
master_map0_skl['share_diff']=master_map0_skl['share_it15']-master_map0_skl['share_b']


In [ ]:
master_map0_skl.to_excel('/Users/hannahkamen/Downloads/map_data_skl.xlsx')

In [ ]:
state_coords['id']= np.arange(len(state_coords))
state_coords['name']=state_coords['state']


In [ ]:
state_coords['lat']=state_coords['centroid2'].apply(lambda x: x.split(',')[1].replace(")",'').strip())
state_coords['lon']=state_coords['centroid2'].apply(lambda x: x.split(',')[0].replace("(",'').replace("c",'').strip())

In [ ]:
state_coords_lm=state_coords[['id','name','lat','lon']]

In [ ]:
state_coords_lm.to_csv('/Users/hannahkamen/Downloads/flowmap_location_lookup.csv',index=False)

In [ ]:
######get total change in population by state 
tmp['total_pct_change_fromb']=(tmp['share_it12']-tmp['share_b'])/tmp['share_b']
total_skilled_pop_changes=tmp.sort_values(by='total_pct_change_fromb',ascending=False)[['statefip','educ_id','age_id','total_pct_change_fromb']]

####merge this on to "move to state" in master_map

master_map0_m=master_map0.merge(total_skilled_pop_changes,left_on='moving_to',right_on='statefip')


#get top entrance and top exit states
top_entrance=list(tmp.sort_values(by='total_pct_change_fromb',ascending=False).head(10)['statefip'])
top_exit=list(tmp.sort_values(by='total_pct_change_fromb',ascending=True).head(10)['statefip'])


In [ ]:
#####prepare pop map data for skilled entrance
all_skl_entrance=pd.DataFrame()
for s in top_entrance:
    ###for each entrance state, calculate shares by origin state that make up total change in the positive share diff
    master_map_lm=master_map0_m[((master_map0_m['moving_to']==s) & (master_map0_m['share_diff']>0))]
    #####get totals 
    master_map_lm_totalinflows=master_map_lm.groupby('moving_to').agg({'share_diff':sum})
    ###rename and merge
    master_map_lm_totalinflows=master_map_lm_totalinflows.rename(columns={'share_diff':'share_diff_tot'})
    master_map_lm=master_map_lm.merge(master_map_lm_totalinflows,on='moving_to')
    
    master_map_lm['inflow_share']= master_map_lm['share_diff']/master_map_lm['share_diff_tot']
    
    master_map_lm=master_map_lm[['moving_to','living_flag','inflow_share','share_diff_tot']]
    all_skl_entrance=all_skl_entrance.append(master_map_lm)

In [ ]:
##merge in pop info


all_skl_entrance=all_skl_entrance.merge(sl,left_on='moving_to',right_on='state')
all_skl_entrance=all_skl_entrance.rename(columns={'abbrev':'dest'})

all_skl_entrance=all_skl_entrance.merge(sl[['state','abbrev']],left_on='living_flag',right_on='state')
all_skl_entrance=all_skl_entrance.rename(columns={'abbrev':'origin'})

all_skl_entrance=all_skl_entrance.merge(state_coords_lm,left_on='dest',right_on='name')
del all_skl_entrance['dest']
all_skl_entrance['dest']=all_skl_entrance['id']
del all_skl_entrance['id']
all_skl_entrance=all_skl_entrance.merge(state_coords_lm,left_on='origin',right_on='name')
del all_skl_entrance['origin']


all_skl_entrance=all_skl_entrance.merge(state_educ,left_on='moving_to',right_on='state').merge(total_skilled_pop_changes,left_on='moving_to',right_on='statefip')
all_skl_entrance['raw_pop_change_moving_to']=all_skl_entrance['total_pct_change_fromb']*all_skl_entrance['state_pop']*all_skl_entrance['skl']*all_skl_entrance['inflow_share']

all_skl_entrance['origin']=all_skl_entrance['id']
all_skl_entrance['count']=all_skl_entrance['raw_pop_change_moving_to']

In [ ]:
all_skl_entrance_lm=all_skl_entrance[['origin','dest','count']]

all_skl_entrance_lm.to_csv('/Users/hannahkamen/Downloads/skilled_entrance.csv',index=False)


In [ ]:
master_map_ks=master_map_lm.merge(sl,left_on='moving_to',right_on='state')
master_map_ks=master_map_ks.rename(columns={'abbrev':'moving_to_abbrev'})
del master_map_ks['state']
master_map_ks=master_map_ks.merge(sl[['state','abbrev']],left_on='living_flag',right_on='state')
master_map_ks=master_map_ks.rename(columns={'abbrev':'living_abbrev'})
del master_map_ks['state']
del master_map_ks['statefip']

master_map_ks=master_map_ks.merge(state_coords,left_on='moving_to_abbrev',right_on='state')
master_map_ks=master_map_ks.merge(state_coords,left_on='living_abbrev',right_on='state')
master_map_ks=master_map_ks.rename(columns={'centroid2_x':'centroid2','centroid2_y':'centroid1','FULL2_NAME_x':'FULL1_NAME','FULL2_NAME_y':'FULL2_NAME'})
                      
                                  

In [ ]:
master_map_ks

In [ ]:
master_map_ks.to_csv('/Users/hannahkamen/Downloads/ks_in_skl_map_data.csv')

In [ ]:
list(master_map_ks)

In [ ]:
master_map_ks

In [ ]:
master_map0_totalinflows=master_map0.groupby('moving_to').agg({'share_diff':sum})
master_map0_totalinflows=master_map0_totalinflows.rename(columns={'share_diff':'share_diff_tot'})
master_map0_m=master_map0.merge(master_map0_totalinflows,on='moving_to')
master_map0_totalinflows=master_map0_totalinflows.reset_index()

In [ ]:
master_map0_m=master_map0_m[['moving_to','living_flag','share_diff','share_diff_tot']]

In [ ]:
master_map0_m

In [ ]:
map_dta=pd.read_csv('/Users/hannahkamen/Downloads/census_texas.csv')

In [ ]:
map_dta_lm=map_dta[map_dta['GEOID2'].str.contains(',')]

map_dta_lm['state']=map_dta_lm['GEOID2'].apply(lambda x: x.split(',')[1])
map_dta_lm['state']=map_dta_lm['GEOID2'].apply(lambda x: x.split(',')[1])

In [ ]:
map_dta_lm

In [ ]:
map_dta['state']

In [ ]:
list(map_dta['GEOID2'])

In [ ]:
master_map0.head()

In [ ]:
##test to see what is happening with temperature

In [ ]:
# df_tmp[df_tmp['state']=='washington']

In [ ]:
master_map.sort_values(by='share',ascending=False).head(100)

In [ ]:
master_map.head()

In [ ]:
df_lm

In [ ]:
master_map.head()

In [ ]:
df_test_gre

In [ ]:
df_tmp

In [ ]:
df_test_gre

In [ ]:
# df_m[(df_m['id']==3)& (df_m['state']=='california')][['msa','bpl','chosen','d_s','d_r1','d_r2','statefip','state']]




In [ ]:
df_test_gre['diff']=abs(df_test_gre['share']-df_test_gre['chosen'])

In [ ]:
df_test_gre['diff'].describe()

In [ ]:
# df_tmp=df_m.groupby(['msa'],as_index=False).agg({'fexthot_28':sum,'fextcold':sum,'state':max,'hot':sum,'cold':sum})

In [ ]:
# df_m[['msa','state','hot','cold','fexthot_28','fextcold']].head()

In [ ]:
len(df[(df['id']==4100)]['msa'].unique())

In [ ]:
list()

In [ ]:
logit_ready[logit_ready['chosen']==1]['d_s'].describe()

In [ ]:
df_lm=df[['msa','chosen','share']]

In [ ]:
df[df['chosen']==1]['share'].describe()

#### Import total population and process percent changes for Windc counterfactual

In [ ]:
#orig=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/dta/dta/acs5yr_0610_clr.dta')

#orig.groupby(['ageid'],as_index=False)

In [ ]:
###get shares by college graduates#impo
#educ_shares.to_csv('/Users/hannahkamen/Downloads/le0_shr2.csv',index=False)
# df['pct_col']=np.exp(df['lncoll'])
# df['pct_nc']=1-np.exp(df['lncoll'])
# df['coll_pop']=np.exp(df['lnpop'])*df['pct_col']
# df['population']=np.exp(df['lnpop'])

# df_tot=df.groupby(['statefip'],as_index=False).agg({'population':sum,'coll_pop':sum,'share':sum,'chosen':sum})
# df_tot['state_coll_shr']=df_tot['coll_pop']/df_tot['population']
# df_tot['pct_change']=(df_tot['share']-df_tot['chosen'])/df_tot['chosen']


In [ ]:
df[df['share'].isnull()]

In [ ]:
df

In [ ]:
list(df)

In [ ]:


df_st=df.groupby('statefip',as_index=False).agg({'share':sum,'population':sum,'chosen':sum,'id':'count'})
df_st['chosen'].sum()

In [ ]:
list(df)

In [ ]:
len(df['id'].unique())

In [ ]:
df_st.head()

In [ ]:
len(df_st['id'].unique())

In [ ]:
44*265

In [ ]:
df[df['id']==62973.0]

In [ ]:
len(df)

In [ ]:
df_st.head()

In [ ]:
len(df['statefip'].unique())

In [ ]:
df.groupby('id',as_index=False).agg({'share':sum})['share'].unique()

In [ ]:
df['msa'].unique()

In [ ]:
len(list(df['msa'].unique()))

In [ ]:
df.head()

In [ ]:
data.head()

In [ ]:
st_lkup=data.groupby(['statefip','str_statefip'],as_index=False).sum()[['statefip','str_statefip']]

In [ ]:
list(data)

In [ ]:
data['str_statefip'].unique()

In [ ]:
list(data)

In [ ]:
st_lkup

In [ ]:
data.head()

In [ ]:
len(data)

In [ ]:
list(data)

In [ ]:
len(logit)

In [ ]:
list(logit)

In [ ]:
logit.head()

In [ ]:
q

In [ ]:
logit.head()

In [ ]:
q.head()

In [ ]:
for i in np.arange(2,8,1):
    print(i)

In [ ]:
for i in np.arange(2,8,1):
    df=pd.read_stata('/Users/hannahkamen/Downloads/population-migration-master/estimation/1_main_specification/acs5yr0610/results/temp/2nd_stage_avg_age%s.dta'%i)
    




In [ ]:
df

In [ ]:
secondndstage2.head()

In [ ]:
secondndstage3.head()

In [ ]:
logit['_b_d_r1'].unique()

In [ ]:
logit['d_s'].unique()

In [ ]:
len(cols)

In [ ]:
cols=[x for x in logit.columns if "_b" in x]

In [ ]:
for x in [x for x in logit.columns if "_b" in x]:
    
    print(logit[x].unique())

In [ ]:
logit['d_s']